In [80]:
import pandas as pd
import os
import json
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("combined_df.csv")
df.shape

In [ ]:
df['scenario'].value_counts()

In [ ]:
df.head()

1. Suppose each simulation run has a result of accident/non-accident, calculate the 
probability of accident (counts, marginal probability). Hint: for each run, the 
collision results are stored in ‘route_highway.txt’. You can check the accident 
status by looking at the ‘status’ field under the ‘record’ section (‘Completed’ means 
no accident; ‘Failed’ means an accident has occurred). (1 point) 

In [ ]:
scenario_status = df.groupby('scenario')['status'].nunique()
scenario_status

In [ ]:
total_runs = df.drop_duplicates(subset=["scenario"]).shape[0]
accident_runs = df[df['status'] == 'Failed'].drop_duplicates(subset=["scenario"]).shape[0]
print(f"Total number of distinct simulation runs: {total_runs}")
print(f"Number of accident runs: {accident_runs}")
accident_probability = accident_runs / total_runs
print(f"Accident probability: {accident_probability:.4f}")

2. By looking at the completion records and the plots you generated in Task 1, under
- which weather condition(s) did the accident happen?
- Does that match your guess in Task 1?
- When did the accident happen during those simulation runs?
- Why do you think the accident happened at that instance? Discuss each accident case separately. (2 points)

In [86]:
pd.set_option('display.max_columns', None)
df['normalized_ts'] = df.groupby('scenario')['ts'].transform(lambda x: x - x.min())

In [87]:
rain_noon_df = df[df['scenario']=='rain-noon']

In [ ]:

plt.figure(figsize=(10, 6))

plt.plot(rain_noon_df['normalized_ts'], rain_noon_df['brake'], label='Brake', color='red')

# speed(v)
# plt.plot(rain_noon_df['normalized_ts'], rain_noon_df['v'], label='Speed (v)', color='blue')

# (throttle)
plt.plot(rain_noon_df['normalized_ts'], rain_noon_df['throttle'], label='Throttle', color='green')

plt.title("Rain-Noon Scenario: Time (Normalized) vs Brake, Speed, and Throttle")
plt.xlabel("Normalized Time")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
rain_noon_df.tail(3)

In [ ]:
accident_brake_data = rain_noon_df[rain_noon_df['brake'] == 1]
accident_brake_data.head()

In [ ]:
filtered_rain_noon_df = rain_noon_df[(rain_noon_df['normalized_ts'] >= 370) & (rain_noon_df['normalized_ts'] <= 400)]
plt.figure(figsize=(10, 6))

plt.plot(filtered_rain_noon_df['ts'], filtered_rain_noon_df['brake'], label='Brake', color='blue')
# plt.plot(filtered_rain_noon_df['ts'], filtered_rain_noon_df['v'], label='Speed (v)', color='blue')
plt.plot(filtered_rain_noon_df['ts'], filtered_rain_noon_df['throttle'], label='Throttle', color='green')
plt.axvline(x=14572, color='red', linestyle='--', label='ts = 14572')

plt.title("Rain-Noon Scenario: Time (ts) vs Brake, Speed, and Throttle")
plt.xlabel("Time (ts)")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
filtered_rain_noon_df = rain_noon_df[(rain_noon_df['normalized_ts'] >= 375) & (rain_noon_df['normalized_ts'] <= 400)]

plt.figure(figsize=(10, 6))

plt.plot(filtered_rain_noon_df['normalized_ts'], filtered_rain_noon_df['brake'], label='Brake', color='red')
plt.plot(filtered_rain_noon_df['normalized_ts'], filtered_rain_noon_df['v'], label='Speed (v)', color='blue')
plt.plot(filtered_rain_noon_df['normalized_ts'], filtered_rain_noon_df['throttle'], label='Throttle', color='green')

plt.title("Rain-Noon Scenario: Time (375-400, Normalized) vs Brake, Speed, and Throttle")
plt.xlabel("Normalized Time (375 - 400)")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
df.iloc[4260:4280]

-----

In [94]:
import matplotlib.pyplot as plt
features = ['throttle', 'steer', 'brake', 'cvip', 'x', 'y', 'v']

In [ ]:
filtered_df = df[(df['normalized_ts'] >= 375) & (df['normalized_ts'] <= 400)]

# Create a plot for each feature with 'normalized_ts' as the x-axis and different scenarios as different lines
for feature in features:
    plt.figure(figsize=(10, 6))
    # Plot each scenario as a line
    for scenario in filtered_df['scenario'].unique():
        scenario_data = filtered_df[filtered_df['scenario'] == scenario]
        plt.plot(scenario_data['normalized_ts'], scenario_data[feature], label=scenario)
    plt.title(f'{feature} Over Normalized Time for Different Scenarios')
    plt.xlabel('Normalized Timestamp')
    plt.ylabel(feature)
    plt.legend(title='Scenario')
    plt.grid(True)
    plt.show()

rain-noon, clear-noon, clear-sunset

(我們取出車禍前後的時段來觀察 )由於 rain-noon, clear-noon, clear-sunset 這三個scneario 在x,y 軸上幾乎是相同的，因此拿出來討論。
注意，這裡我們有將ts 標準化，也就是讓每一個scneario的起始點變成是0
當車禍要發生時，rain-noon首先比其他兩個scenario晚踩煞車，然後它的煞車也是斷斷續續，猜測有可能是系統沒設定好，導致最後速度並沒有像是其他兩個scenario一樣把速度減下來(從下方的v圖可以看出)

In [ ]:
# scenarios_to_analyze = ['rain-noon', 'clear-noon', 'clear-sunset']
# filtered_df = df[df['scenario'].isin(scenarios_to_analyze)]
filtered_df = filtered_df[(filtered_df['normalized_ts'] >= 375) & (df['normalized_ts'] <= 400)]

features = ['brake', 'v', 'throttle', 'cvip', 'x','y']

for feature in features:
    plt.figure(figsize=(10, 6))
    for scenario in filtered_df['scenario'].unique():
        scenario_data = filtered_df[filtered_df['scenario'] == scenario]
        plt.plot(scenario_data['normalized_ts'], scenario_data[feature], label=scenario)
    plt.title(f'{feature} Over Normalized Time for Different Scenarios')
    plt.xlabel('Normalized Timestamp')
    plt.ylabel(feature)
    plt.axvline(x=382, color='red', linestyle='--', label='accident ts')
    plt.legend(title='Scenario')
    plt.grid(True)
    plt.show()


In [ ]:
for feature in features:
    plt.figure(figsize=(10, 6))
    # Plot each scenario as a line
    for scenario in df['scenario'].unique():
        scenario_data = df[df['scenario'] == scenario]
        plt.plot(scenario_data['normalized_ts'], scenario_data[feature], label=scenario)
    plt.title(f'{feature} Over Normalized Time for Different Scenarios')
    plt.xlabel('Normalized Timestamp')
    plt.ylabel(feature)
    plt.legend(title='Scenario')
    plt.grid(True)
    plt.show()

In [ ]:
# Limit the plot to normalized time greater than 10 seconds
feature = 'cvip'

plt.figure(figsize=(10, 6))
# Plot each scenario as a line, restricting to normalized_ts > 10
for scenario in df['scenario'].unique():
    scenario_data = df[(df['scenario'] == scenario) & (df['normalized_ts'] > 10)]
    plt.plot(scenario_data['normalized_ts'], scenario_data[feature], label=scenario)

plt.title(f'{feature} Over Normalized Time (After 10 seconds) for Different Scenarios')
plt.xlabel('Normalized Timestamp')
plt.ylabel(feature)
plt.legend(title='Scenario')
plt.grid(True)
plt.show()


In [ ]:
# Limit the plot to normalized time between 350 and 450 seconds
for feature in features:
    plt.figure(figsize=(10, 6))
    # Plot each scenario as a line, restricting to normalized_ts between 350 and 450
    for scenario in df['scenario'].unique():
        scenario_data = df[(df['scenario'] == scenario) &
                                    (df['normalized_ts'] >= 360) &
                                    (df['normalized_ts'] <= 450)]
        plt.plot(scenario_data['normalized_ts'], scenario_data[feature], label=scenario)
    plt.title(f'{feature} Over Normalized Time (350-450 seconds) for Different Scenarios')
    plt.xlabel('Normalized Timestamp')
    plt.ylabel(feature)
    plt.legend(title='Scenario')
    plt.grid(True)
    plt.show()


In [ ]:
selected_scenarios = ['clear-noon', 'clear-sunset', 'rain-noon']
for feature in features:
    plt.figure(figsize=(10, 6))
    # Plot each selected scenario as a line, restricting to normalized_ts between 350 and 450
    for scenario in selected_scenarios:
        scenario_data = df[(df['scenario'] == scenario) &
                                    (df['normalized_ts'] >= 360) &
                                    (df['normalized_ts'] <= 450)]
        plt.plot(scenario_data['normalized_ts'], scenario_data[feature], label=scenario)
    
    plt.title(f'{feature} Over Normalized Time (360-450 seconds) for Selected Scenarios')
    plt.xlabel('Normalized Timestamp')
    plt.ylabel(feature)
    plt.legend(title='Scenario')
    plt.grid(True)
    plt.show()

---

### Q4.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Separate normal and abnormal runs based on 'status' column
normal_runs = df[df['status'] == 'Completed']
abnormal_runs = df[df['status'] != 'Completed']

# List of features to study
features = ["brake", "steer", "v", "y", "x", "cvip", "throttle"]

# Plot density distribution for each feature
plt.figure(figsize=(15, 12))
for i, feature in enumerate(features):
    plt.subplot(3, 3, i + 1)
    sns.kdeplot(normal_runs[feature], label='Normal', color='green')
    sns.kdeplot(abnormal_runs[feature], label='Abnormal', color='red')
    plt.title(f'Distribution of {feature}')
    plt.xlabel(feature)
    plt.ylabel('Density')
    plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Descriptive analysis for 'steer' feature
steer_normal_mean = normal_runs['steer'].mean()
steer_abnormal_mean = abnormal_runs['steer'].mean()
steer_normal_std = normal_runs['steer'].std()
steer_abnormal_std = abnormal_runs['steer'].std()

# Print descriptive analysis
steer_description = {
    'normal_mean': steer_normal_mean,
    'abnormal_mean': steer_abnormal_mean,
    'normal_std': steer_normal_std,
    'abnormal_std': steer_abnormal_std
}

steer_description

In [ ]:
# Check variance equality
var_equal = stats.levene(normal_runs['steer'], abnormal_runs['steer']).pvalue > 0.05

# Perform two-sample t-test
t_stat, p_value = stats.ttest_ind(normal_runs['steer'], abnormal_runs['steer'], equal_var=var_equal)

# Determine if we reject the null hypothesis at 0.05 significance level
reject_null = p_value < 0.05

# Return test results
t_test_results = {
    't_statistic': t_stat,
    'p_value': p_value,
    'variance_equal': var_equal,
    'reject_null': reject_null
}

t_test_results


### Q5.

In [ ]:
# Calculate the Pearson correlation coefficient between the identified features
correlation_matrix = df[['steer', 'v', 'cvip']].corr()

# Display the correlation matrix
correlation_matrix


### Q6. 

In [ ]:
# Perform Kolmogorov-Smirnov two-sample test for each feature
ks_results = {}

# Perform KS test for 'steer'
ks_stat_steer, ks_pvalue_steer = stats.ks_2samp(normal_runs['steer'], abnormal_runs['steer'])
ks_results['steer'] = {'ks_statistic': ks_stat_steer, 'p_value': ks_pvalue_steer}

# Perform KS test for 'v'
ks_stat_v, ks_pvalue_v = stats.ks_2samp(normal_runs['v'], abnormal_runs['v'])
ks_results['v'] = {'ks_statistic': ks_stat_v, 'p_value': ks_pvalue_v}

# Perform KS test for 'cvip'
ks_stat_cvip, ks_pvalue_cvip = stats.ks_2samp(normal_runs['cvip'], abnormal_runs['cvip'])
ks_results['cvip'] = {'ks_statistic': ks_stat_cvip, 'p_value': ks_pvalue_cvip}

ks_results

In [ ]:
# Perform Kolmogorov-Smirnov two-sample test for the features not selected as indicators
not_selected_features = ['brake', 'y', 'x', 'throttle']
ks_results_not_selected = {}

for feature in not_selected_features:
    ks_stat, ks_pvalue = stats.ks_2samp(normal_runs[feature], abnormal_runs[feature])
    ks_results_not_selected[feature] = {'ks_statistic': ks_stat, 'p_value': ks_pvalue}

# Display the results
ks_results_not_selected

### Q8. 

In [111]:
# !pip install dtaidistance

In [ ]:
from dtaidistance import dtw

# 提取三個不同場景的 steering 數據：clear-noon（基準）、clear-night 和 clear-sunset
steer_clear_noon = df[df['scenario'] == 'clear-noon']['steer'].values
steer_clear_night = df[df['scenario'] == 'clear-night']['steer'].values
steer_clear_sunset = df[df['scenario'] == 'clear-sunset']['steer'].values

# 計算 DTW 距離
dtw_distance_night = dtw.distance(steer_clear_noon, steer_clear_night)
dtw_distance_sunset = dtw.distance(steer_clear_noon, steer_clear_sunset)

# 顯示 DTW 距離
print(f"DTW Distance (Clear-Noon vs Clear-Night): {dtw_distance_night}")
print(f"DTW Distance (Clear-Noon vs Clear-Sunset): {dtw_distance_sunset}")


----

(my thought)

In [ ]:
# Normalizing the 'ts' column within each 'scenario'
df['normalized_ts'] = df.groupby('scenario')['ts'].transform(lambda x: x - x.min())
filtered_df = df[df['normalized_ts'] <= 401]
# filtered_df.head()
filtered_df['scenario'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Separate normal and abnormal runs based on 'status' column
normal_runs = filtered_df[filtered_df['status'] == 'Completed']
abnormal_runs = filtered_df[filtered_df['status'] != 'Completed']

# List of features to study
features = ["brake", "steer", "v", "y", "x", "cvip", "throttle"]

# Plot density distribution for each feature
plt.figure(figsize=(15, 12))
for i, feature in enumerate(features):
    plt.subplot(3, 3, i + 1)
    sns.kdeplot(normal_runs[feature], label='Normal', color='green')
    sns.kdeplot(abnormal_runs[feature], label='Abnormal', color='red')
    plt.title(f'Distribution of {feature}')
    plt.xlabel(feature)
    plt.ylabel('Density')
    plt.legend()

plt.tight_layout()
plt.show()